In [29]:
from codeinterpreter.sandbox.local_jupyter.manager import LocalJupyterManager

In [2]:
manager = LocalJupyterManager()
manager.init()
sandbox =  manager.start()

In [3]:
def code_run(code:str):
    output =  sandbox.run(code=code)
    return output.content

In [4]:
from langchain.tools import StructuredTool

In [33]:
from pydantic import BaseModel
class CodeInput(BaseModel):
    code: str


In [24]:
tools = [
    StructuredTool(
        name="python",
        description="Input a string of code to a ipython interpreter. "
        "Write the entire code in a single string. This string can "
        "be really long, so you can use the `;` character to split lines. "
        "Variables are preserved between runs. ",
        func=code_run,
        args_schema=CodeInput,
    )
]

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [34]:
agent.run("pick a color to print ! red,blue,yellow")




> Entering new AgentExecutor chain...

Invoking: `python` with `{'code': "print('Blue')"}`
responded: {content}

BlueThe color I picked to print is blue.

> Finished chain.


'The color I picked to print is blue.'